In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

## 1. Load the Dataset (Using PlantVillage as a substitute)
print("Loading PlantVillage dataset...")
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

NUM_CLASSES = ds_info.features['label'].num_classes
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
## 2. Preprocessing Function
# This resizes images and scales pixel values to what MobileNetV2 expects.
def preprocess_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess_image).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_image).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_image).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

print(f"Dataset loaded with {NUM_CLASSES} classes.")

In [ ]:
## 3. Build the Model (Simulating MobileNetV2)
print("Building model with MobileNetV2 base...")
# Load MobileNetV2 pre-trained on ImageNet, without its top classification layer
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model layers to leverage pre-trained features
base_model.trainable = False

# Create a new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

In [ ]:

## 4. Compile and Train the Model
# We use Adam optimizer and SparseCategoricalCrossentropy for multi-class classification
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTraining the model...")
history = model.fit(
    train_ds,
    epochs=5, # Using 5 for demonstration; more epochs needed for higher accuracy
    validation_data=val_ds
)

In [ ]:

## 5. Evaluate the Model
print("\nEvaluating on the test set...")
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the trained model for the next step
model.save('crop_disease_classifier.h5')
print("\nModel saved as crop_disease_classifier.h5")